### 6. Feladatsor

Konzisztencia rend, stabilitás 1.

#### Dahlquist-féle tesztegyenlet

$$
\begin{align}
     \dot x = \lambda x
\end{align}
$$

ahol $\lambda \in \mathbb{C}$ paraméter. 


Megoldása: 

$$x(t) = c\cdot e^{\lambda t} = c\cdot e^{(a+bi)t} = c\cdot e^{a t} e^{ibt}.$$

Ennek hossza
$$ |x(t)| = |c| \cdot e^{at},$$

mely függvény monotonicitását $a$ előjele dönti el.

#### 1. Feladat

Tekintsük a feladatot a $[0,1]$ intervallumon $\lambda = 1$ paraméter, $x(0) = 1$ kezdetiérték mellett.

a) Oldjuk meg az EE módszer segítségével, 
$$h = 10^{-k} \quad (k=2, \ldots, 5)$$ 
lépéshosszokkal.

In [ ]:
function [ts, ys] = explicit_euler(f, ts, y0)
    ys = zeros(numel(y0), numel(ts));
    ys(:, 1) = y0;
    
    for k=1:(numel(ts) - 1)
        h = ts(k+1) - ts(k);
        ys(:, k+1) = ys(:, k) + h * f(ts(k), ys(:, k));
    end
end

In [ ]:
h = 10^(-5);
[ts, ys] = explicit_euler(@(t, x)(x), 0:h:1, 1);

b) Megoldásainkat hasonlítsuk össsze a pontos megoldással, a végeredmény legyen a hibavektorok (valamilyen normában mért) hossza.

c) Az így kapott hibákat ábrázoljuk egy log-log ábrán.

d) Hogy tudnánk a kapott adatokra egyenest illeszteni? Ránézésre mi lenne ennek a meredeksége?

#### 2. Feladat

Mi történik amikor az EE módszert alkalmazzuk a tesztegyenletre?

$$
    y_{n+1} = y_n + h f(t_n, y_n) = y_n + h\lambda y_n = (1 + z)y_n = R(z)y_n,
$$

ahol $\mathbb{C} \ni z = h\lambda$ komplex szám egyszerre modellezi a feladatot és a lépéshosszt, az $R(z)$ stabilitási-függvény pedig a numerikus módszert.

Explicit módszer stabilitási-függvénye mindig polinom.

A (lineáris) rendfeltételeket teljesítő módszerekre
$$
R(z) \approx e^z,
$$
ahol a közelítés rendje megegyezik a módszer rendjével.

Szükségszerűen az explicit módszerek stabilitási-függvényeinek első néhány tagja $e^z$ hatványsorának első néhány tagjával fog megegyezni. Például:
$$
    1 + z + \frac{1}{2} z^2 \approx e^z.
$$

Tudjuk, hogy ha $\operatorname{Re}(\lambda) \leq 0$, akkor a pontos megoldás hossza időben monoton csökkenő. Igaz-e ez a numerikus megoldásra?

$$
 | y_{n+1} | = |R(z)| |y_n| \leq |y_n|,
$$

így világos, hogy a megfelelő feltétel a
$$
 |R(z)| \leq 1.
$$

In [ ]:
[X, Y] = meshgrid( linspace(-3, 3, 40), linspace(-3, 3, 40));
Z = X + 1i*Y;

a) Az EE módszer stabilitási függvénye $1+z$. Ábrázoljuk a stabilitási tartományát.

In [ ]:
% EE
contourf(X, Y, abs(1 + Z), linspace(0, 1, 4), 'ShowText','on')
colorbar
grid on
colormap summer

b) A MPM stabilitási függvénye $1+z + \frac{1}{2} z^2$. Ábrázoljuk a stabilitási tartományát.

In [ ]:
% MPM
contourf(X, Y, abs(1 + Z + .5*Z.^2), [0, .5, 1], 'ShowText','on')
colorbar
grid on
colormap summer

c) Az IE módszer stabilitási függvénye $\frac{1}{1-z}$. Ábrázoljuk a stabilitási tartományát.

In [ ]:
% IE
contourf(X, Y, 1./abs(1-Z), [0, 1], 'ShowText','on')
colorbar
colormap summer
grid on

#### 3.* Feladat

Írjunk függvényt, ami Butcher-tablója $(A,b)$ alapján ábrázolja a Runge-Kutta módszer stabilitási-tartományát.
Felhasználhatjuk a

$$
R(z) = 1 + zb^T (I - zA)^{-1} e
$$

képletet, ahol $b$ oszlopvektor, $I$ egységmátrix, $e = [1, \ldots, 1]^T$.

In [ ]:
function visualize_stability_area(A, b, xlims, ylims, ticks)
    tic;
    
    [X, Y] = meshgrid(linspace(xlims(1), xlims(2), ticks),
                      linspace(ylims(1), ylims(2), ticks));
    Z = X + 1i*Y;

    vecZ = Z(:);
    
    I  = eye(rows(A)*numel(vecZ));
    e  = ones(rows(A)*numel(vecZ), 1);
    bT = kron(eye(numel(vecZ)), b);
        
    Rz = 1 + vecZ .*  bT * ((I - kron(diag(vecZ), A))  \ e);

    contourf(X, Y, abs(reshape(Rz, size(Z))), [0, .25, .5, .75 1], 'ShowText','on')
    colorbar
    grid on
    colormap summer
    toc;
end

In [ ]:
% RK4

A = [ 0, 0, 0, 0 
     .5, 0, 0, 0
      0,.5, 0, 0
      0, 0, 1, 0];
b = [ 1, 2, 2, 1]/6;

visualize_stability_area(sparse(A), b, [-3 1], [-3 3], 40)